In [4]:
import numpy as np
from scipy.io import loadmat
from scipy.stats import gamma
from scipy.special import digamma, loggamma
import configs

# Helper functions

In [5]:
def get_kl_nfields(nfieldsA, nfieldsB):                                                       
    """ Get KL divergence of nfields distribution from Payne 2021 distribution. """
                                                                                   
    nfieldsA = nfieldsA[nfieldsA != 0].copy()
    nfieldsB = nfieldsB[nfieldsB != 0].copy()
    nfieldsA[nfieldsA > 5] = 5 # Bins are [1, 2, 3, 4, 5+]   
    nfieldsB[nfieldsB > 5] = 5 # Bins are [1, 2, 3, 4, 5+]   
    P = np.array([                                                                 
        np.sum(nfieldsA==num)/nfieldsA.size for num in np.arange(1, 6)               
        ])                                                                         
    Q = np.array([                                                                 
        np.sum(nfieldsB==num)/nfieldsB.size for num in np.arange(1, 6)               
        ])                                      
                                                                                   
    kl = 0                                                                         
    for idx in np.arange(Q.size):                                                  
        p_x = P[idx]; q_x = Q[idx]
        if p_x == 0: continue                                                      
        kl += p_x * np.log(p_x/(q_x+1E-5))
    return kl                                                                      
                                                                                   
def get_kl_fieldsizes(fieldsizesA, fieldsizesB):                                                 
    """                                                                            
    Get KL divergence of field size distribution from Payne 2021 distribution.  
    """                                                                            
                                                                                   
    k_P, _, theta_P = gamma.fit(fieldsizesA) # shape, scale                         
    k_Q, _, theta_Q = gamma.fit(fieldsizesB) # shape, scale         
    term1 = (k_P - k_Q) * digamma(k_P)                                             
    term2 = -loggamma(k_P)                                                         
    term3 = loggamma(k_Q)                                                          
    term4 = k_Q * (np.log(theta_Q) - np.log(theta_P))                              
    term5 = k_P * (theta_P - theta_Q)/theta_Q                                      
    kl = term1 + term2 + term3 + term4 + term5                                     
    return kl 

# Load data

In [6]:
path_to_payne2021 = '/Users/chingfang/Downloads/6bc_payne2021_data.mat'
data = loadmat(path_to_payne2021)

In [7]:
field_ratios = np.array(data['field_ratios']).squeeze()

In [8]:
field_nums = np.array(data['field_nums']).squeeze()

# Get split halves

In [9]:
field_nums_base = []
field_ratios_base = []
for _ in range(500):
    nfieldsA = np.random.choice(field_nums, size=field_nums.size//2)
    nfieldsB = np.random.choice(field_nums, size=field_nums.size//2)
    fieldsizesA = np.random.choice(field_ratios, size=field_ratios.size//2)
    fieldsizesB = np.random.choice(field_ratios, size=field_ratios.size//2)
    field_nums_base.append(get_kl_nfields(nfieldsA, nfieldsB))
    field_ratios_base.append(get_kl_fieldsizes(fieldsizesA, fieldsizesB))

In [15]:
# KL Field Nums
print(f'Mean: {np.mean(field_nums_base)}')
print(f'Std: {np.std(field_nums_base)}')

Mean: 0.03671326596653946
Std: 0.03038093819129283


In [16]:
# KL Field Sizes
print(f'Mean: {np.mean(field_ratios_base)}')
print(f'Std: {np.std(field_ratios_base)}')

Mean: 0.08870356620554733
Std: 0.10243461029897821


In [18]:
# Sum D_KL
kls = np.array(field_nums_base) + np.array(field_ratios_base)
print(f'Mean: {np.mean(kls)}')
print(f'Std: {np.std(kls)}')

Mean: 0.1254168321720868
Std: 0.10782275275913869
